In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp loggers.core

In [4]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from torch.multiprocessing import Queue
import torchdata.datapipes as dp
# Local modules
from fastrl.pipes.core import *

# Loggers Core
> Core Utilties for logging in fastrl using Callbacks and DataPipes

In [5]:
# export
class LoggerBase(Callback):
    def __init__(self):
        self.main_queue = Queue()

The `LoggerBase` class outlines simply the `main_queue`. It works in combo with `LogCollector` datapipe which will add to the `main_queue` even across processes. 

In [6]:
# export
class LogCollector(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,logger_base:LoggerBase):
        self.source_datapipe = source_datapipe
        self.main_queue = logger_base.main_queue
        
    def __iter__(self): raise NotImplementedError

Notes:

User can init multiple different logger bases if they want

We then can manually add Collectors, custom for certain pipes such as for collecting rewards. This can be put in 
full on pipelines even across workers.

The manual process might not be desireable though, so I'm thinking of revising the `add_cbs_to_pipes` where
the pipe itself literally says if it should go before, after, exclude from.

This seems... easier than defining a callback, and keeps a level of flatness to the api.





In [ ]:
# export
class ProgressBarLogger(LoggerBase):
    def add_reward_logger(self,before=dp.iter.Batcher,after=None,not_under=None) -> List[dp.iter.IterDataPipe]:
        return L(OneAdder)

In [8]:
import pandas as pd
from fastrl.envs.gym import *

envs = ['CartPole-v1']*10

pipe = dp.map.Mapper(envs)
pipe = TypeTransformLoop(pipe,[GymTypeTransform])
pipe = dp.iter.MapToIterConverter(pipe)
pipe = dp.iter.InMemoryCacheHolder(pipe)
pipe = pipe.cycle(count=(18*len(envs))) 
# Turn off the seed so that some envs end before others...
pipe = GymStepper(pipe,synchronized_reset=True)

pipe = add_cbs_to_pipes(pipe,L(cb))

steps = list(pipe)

In [10]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
